In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'IntroductionToNaturalLanguageProcessing/project/github_submission'

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd /content/drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/IntroductionToNaturalLanguageProcessing/project/github_submission


In [2]:
!pip install -U pip
!pip install accelerate==0.20.3
!pip install appdirs==1.4.4
!pip install bitsandbytes==0.37.2
!pip install datasets==2.10.1
!pip install fire==0.5.0
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torch==2.0.1
!pip install sentencepiece==0.1.97
!pip install tensorboardX==2.6
!pip install gradio==3.23.0

!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-7qr3s8qr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-7qr3s8qr
  Resolved https://github.com/huggingface/peft.git to commit 08cb3dde577747f6ca6638c884fd66fd16cf2e9d
 

In [3]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-a100-s-3g

Hyperparameter

In [4]:
GENERATE_CASE = 100
PRINT_CASE = 4

Tokenizer

In [5]:
BASE_MODEL = "decapoda-research/llama-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Prompt

In [6]:
def generate_discriminator_test_prompt(DataFrame, fake=True):
    if fake == True:
        return f"""Below is a piece of review content. Your task is to verify whether the review was written by the critic specified. The response should always be True if the review was written by the critic, or False if it was not.
    ### Instruction:
    Verify if the review content was written by the specified critic. Your response should be either 'True' if it was, or 'False' if it was not.
    ### Input:
    [Critic] {DataFrame["critic_name"].values[0]}
    [Review] {DataFrame["fake_review"].values[0]}
    ### Response:"""

    else:
        return f"""Below is a piece of review content. Your task is to verify whether the review was written by the critic specified. The response should always be True if the review was written by the critic, or False if it was not.
    ### Instruction:
    Verify if the review content was written by the specified critic. Your response should be either 'True' if it was, or 'False' if it was not.
    ### Input:
    [Critic] {DataFrame["critic_name"].values[0]}
    [Review] {DataFrame["review_content"].values[0]}
    ### Response:"""

In [7]:
def generate_test_prompt1(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
[Movie_info] {data_point["movie_info"]}
{data_point["critic_reviews_history"]}
### Response:
"""

def generate_test_prompt2(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
[Movie_info] {data_point["movie_info"]}
[Movie genres]{data_point["genres"]}
[Review score]{data_point["review_score"]}
[review history]{data_point["critic_reviews_history"]}
### Response:
"""

def generate_test_prompt3(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
[Movie_info] {data_point["movie_info"]}
[Movie genres]{data_point["genres"]}
[Review score]{data_point["review_score"]}
[review history]{data_point["critic_reviews_history"]}
### Response:
"""

# Store the prompt generation functions in a dictionary
test_prompt_generators = {
    "model1": generate_test_prompt1,
    "model2": generate_test_prompt2,
    "model3": generate_test_prompt3,
}


Eval Function (Sentiment Score, Cossim)

In [8]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer


def evaluate_sentiment_match(text1, text2):
    sid = SentimentIntensityAnalyzer()

    # Preprocess the text (convert to lowercase)
    text1 = text1.lower()
    text2 = text2.lower()

    # Calculate sentiment scores for each text
    sentiment1 = sid.polarity_scores(text1)
    sentiment2 = sid.polarity_scores(text2)

    # Compare sentiment scores (considering the compound score)
    score1 = sentiment1['compound']
    score2 = sentiment2['compound']

    # Define a matching criterion (e.g., within a range of 0.1)
    match_threshold = 0.1
    sentiment_match = abs(score1 - score2)

    return sentiment_match

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy.linalg import norm
def cossim(A,B):
# define two lists or array
    cosine = np.dot(A,B)/(norm(A)*norm(B))
    return cosine

embeddingmodel = SentenceTransformer('all-mpnet-base-v2')

def evaluate_cossim(responses,real_comments):
  score=0
  for i in range(len(responses)):
    sentences = [responses[i], real_comments[i]]
    embeddings = embeddingmodel.encode(sentences)
    score+=cossim(embeddings[0],embeddings[1])
  return score/len(responses)

Discriminator Dataset

In [10]:
import pandas as pd
from sklearn.utils import shuffle
import random

# get top 5 critics
df = pd.read_csv("data.csv")
critic_counts = df['critic_name'].value_counts()
top_5_critics = critic_counts.nlargest(5).index
df = df[df['critic_name'].isin(top_5_critics)].reset_index(drop=True)

real_reviews = df[['critic_name', 'review_content']]
real_reviews['label'] = 1

fake_reviews = df[['critic_name', 'review_content']].copy()
new_order = []
last_critic_name = None
for _ in range(len(df)):
    available_critics = top_5_critics.tolist()
    if last_critic_name is not None:
        available_critics.remove(last_critic_name)
    chosen_critic = random.choice(available_critics)
    new_order.append(chosen_critic)
    last_critic_name = chosen_critic

fake_reviews['critic_name'] = new_order
fake_reviews['label'] = 0

dataset = pd.concat([real_reviews, fake_reviews])
dataset = shuffle(dataset)

dataset.head()

<ipython-input-10-49bcf51ae49a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_reviews['label'] = 1


,critic_name,review_content,label
16925,Roger Ebert,"The film is aimed at children, I suppose, alth...",1
29024,Roger Moore,Fascinating film that doesn't blemish Gibney's...,1
21584,Roger Ebert,The film begins as an unyielding look at a bat...,1
4283,Frank Swietek,What writer-director Leslye Headland has given...,0
11051,Roger Ebert,Thought-provoking.,1


In [11]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

train_dataset_data = [
    {
        "instruction": "Determine if the review was written by the critic",
        "critic_name": row_dict['critic_name'],
        "review_content": row_dict['review_content'],
        "output": 'True' if row_dict['label'] == 1 else 'False'
    }
    for row_dict in train_data.to_dict(orient="records")
]

test_dataset_data = [
    {
        "instruction": "Determine if the review was written by the critic",
        "critic_name": row_dict['critic_name'],
        "review_content": row_dict['review_content'],
        "output": 'True' if row_dict['label'] == 1 else 'False'
    }
    for row_dict in test_data.to_dict(orient="records")
]

# Test Discriminator

In [12]:
from peft import PeftModel

adapters_name = "experiments/discriminator"
BASE_MODEL = "decapoda-research/llama-7b-hf"

discriminator_model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
discriminator_model = PeftModel.from_pretrained(discriminator_model, adapters_name)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [13]:
#set config for discriminator_model generation
generation_config = discriminator_model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

responses = []
real_comments = []
test_prompts = [generate_discriminator_test_prompt(pd.DataFrame([data_point]), fake=False) for data_point in test_dataset_data]

# test case
for i in range(PRINT_CASE):
    input_ids = tokenizer.encode(test_prompts[i], return_tensors='pt')
    output = discriminator_model.generate(input_ids=input_ids,
                        generation_config=generation_config)
    #decode output
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)

    #Keep the generated comment
    response_start = output_str.find('### Response:\n') + len('### Response:\n')
    response_end = output_str.find('###', response_start)

    if response_end != -1:  # If '### Review:' is found in the output
        response = output_str[response_start:response_end].strip()
    else:
        response = output_str[response_start:].strip()

    responses.append(response)
    real_comments.append(test_dataset_data[i]['output'])
    print(f"Generated Review {i+1}:\n{output_str}\n\n")
    print(f"Real Label:{test_dataset_data[i]['output']}\n")
    print("\n"+"-"*50+"\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated Review 1:
Below is a piece of review content. Your task is to verify whether the review was written by the critic specified. The response should always be True if the review was written by the critic, or False if it was not.
    ### Instruction:
    Verify if the review content was written by the specified critic. Your response should be either 'True' if it was, or 'False' if it was not.
    ### Input:
    [Critic] Dennis Schwartz
    [Review] A decent film in the long-standing series.
    ### Response:
    True
    ### Explanation:
    The review content was written by the specified


Real Label:True


--------------------------------------------------

Generated Review 2:
Below is a piece of review content. Your task is to verify whether the review was written by the critic specified. The response should always be True if the review was written by the critic, or False if it was not.
    ### Instruction:
    Verify if the review content was written by the specified critic. Y

Generator Dataset

In [14]:
def get_reviews(df, critic_name, movie_title):
    critic_reviews = df[df['critic_name'] == critic_name]
    critic_reviews = critic_reviews[critic_reviews['movie_title'] != movie_title]
    reviews = critic_reviews['review_content'].sample(n=min(5, len(critic_reviews)), random_state=42)

    reviews = ['[REVIEW] ' + review for review in reviews]

    return reviews

In [15]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

df = pd.read_csv("data.csv")
critic_counts = df['critic_name'].value_counts()
top_5_critics = critic_counts.nlargest(5).index
df = df[df['critic_name'].isin(top_5_critics)]
df, _ = train_test_split(df, test_size=0.4, random_state=42)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

def create_dataset_entry(df, row_dict):
    return {
        "movie_title" : row_dict['movie_title'],
        "movie_info": row_dict['movie_info'],
        "genres": row_dict.get("genres"),  # Use .get() to avoid KeyError if the key is not present
        "review_score": row_dict.get("review_score"),
        "critic_reviews_history" : get_reviews(df, row_dict['critic_name'], row_dict['movie_title']),
        "output": row_dict['review_content']
    }

train_dataset = [create_dataset_entry(train_data, row) for row in tqdm(train_data.to_dict(orient="records"), desc='Creating Train Dataset')]
test_dataset = [create_dataset_entry(test_data, row) for row in tqdm(test_data.to_dict(orient="records"), desc='Creating Test Dataset')]

instructions = [
    "Make new review",
    "Write a new review for {movie_title}",
    "You are a movie critic, using movie infos and review you have written before, write a new short review for movie {movie_title}"
]

train_test_datasets = {}
for i, instruction in enumerate(instructions):
    train_test_datasets[f'model{i+1}'] = {
        'train': [{**entry, "instruction": instruction.format(movie_title=entry['movie_title'])} for entry in tqdm(train_dataset, desc=f'Creating Training Model {i+1}')],
        'test': [{**entry, "instruction": instruction.format(movie_title=entry['movie_title'])} for entry in tqdm(test_dataset, desc=f'Creating Testing Model {i+1}')],
    }

Creating Testing Model 3: 100%|██████████| 3489/3489 [00:00<00:00, 465855.75it/s]


# Test Generator

In [16]:
def generate_reviews(MODEL, model, test_dataset_data, tokenizer, GENERATE_CASE, PRINT_CASE):
    #set config for model generation
    generation_config = model.generation_config
    generation_config.max_new_tokens = 60
    generation_config.temperature = 0
    generation_config.num_return_sequences = 1
    generation_config.pad_token_id = tokenizer.eos_token_id
    generation_config.eos_token_id = tokenizer.eos_token_id

    responses = []
    real_comments = []
    test_prompts = [test_prompt_generators[MODEL](data_point) for data_point in test_dataset_data]

    # test case
    for i in range(GENERATE_CASE):
        input_ids = tokenizer.encode(test_prompts[i], return_tensors='pt')
        output = model.generate(input_ids=input_ids,
                                generation_config=generation_config)
        #decode output
        output_str = tokenizer.decode(output[0], skip_special_tokens=True)

        #Keep the generated comment
        response_start = output_str.find('### Response:\n') + len('### Response:\n')
        response_end = output_str.find('###', response_start)

        if response_end != -1:  # If '### Review:' is found in the output
            response = output_str[response_start:response_end].strip()
        else:
            response = output_str[response_start:].strip()

        responses.append(response)
        real_comments.append(test_dataset_data[i]['output'])
        if i < PRINT_CASE:
            print(f"Generated Review {i+1}:\n{output_str}\n\n")
            print(f"Real Label:{test_dataset_data[i]['output']}\n")
            print("\n"+"-"*50+"\n")

    return responses, real_comments

# Test Model1

In [17]:
from peft import PeftModel

MODEL = "model1"
adapters_name = f"experiments/generator/{MODEL}"
BASE_MODEL = "decapoda-research/llama-7b-hf"

generator_model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
generator_model = PeftModel.from_pretrained(generator_model, adapters_name)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [18]:
test_dataset_data = train_test_datasets[MODEL]['test']

In [19]:
responses, real_comments = generate_reviews(MODEL, generator_model, test_dataset_data, tokenizer, GENERATE_CASE, PRINT_CASE)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated Review 1:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
Make new review
### Input:
[Movie_info] Gigli (Ben Affleck) is ordered to kidnap the psychologically challenged younger brother of a powerful federal prosecutor. When plans go awry, Gigli's boss sends in Ricki (Jennifer Lopez), a gorgeous free-spirited female gangster who has her own set of orders to assist with the kidnapping. But Gigli begins falling for the decidedly unavailable Ricki, which could be a hazard to his occupation.
["[REVIEW] Has a derivative feel, overly reminiscent not only of fifties Broadway drama but of some of Allen's previous pictures...while it probably won't bore you, it's likely to leave you with a feeling of déjà vu.", "[REVIEW] Astonishingly bad...a would-be comedy that's not just cruelly unfunny but sloppily made.", "[REVIEW] The effect is of a weepy w

EVAL

In [20]:
sentiment_scores = []
for i in range(len(responses)):
    score = evaluate_sentiment_match(real_comments[i],responses[i])
    sentiment_scores.append(score)

print("sentiment_scores:",sum(sentiment_scores) / len(sentiment_scores))
print("cossim:",evaluate_cossim(responses,real_comments))

sentiment_scores: 0.49587799999999993
cossim: 0.17700144685804844


Discriminator(generator)

In [21]:
test_prompts = []
for i in range(PRINT_CASE):
    temp_df = df[df["review_content"] == real_comments[i]]
    temp_df["fake_review"] = responses[i]
    test_prompts.append(generate_discriminator_test_prompt(temp_df, fake=True))

<ipython-input-21-09a91492e7fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["fake_review"] = responses[i]
<ipython-input-21-09a91492e7fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["fake_review"] = responses[i]
<ipython-input-21-09a91492e7fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [22]:
#set config for model generation
generation_config = discriminator_model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

# test case
for i in range(PRINT_CASE):
    input_ids = tokenizer.encode(test_prompts[i], return_tensors='pt')
    output = discriminator_model.generate(input_ids=input_ids,
                        generation_config=generation_config)
    #decode output
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f"Generated Review {i+1}:\n{output_str}\n\n")
    print(f"Real Label:{real_comments[i]}\n")
    print("\n"+"-"*50+"\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated Review 1:
Below is a piece of review content. Your task is to verify whether the review was written by the critic specified. The response should always be True if the review was written by the critic, or False if it was not.
    ### Instruction:
    Verify if the review content was written by the specified critic. Your response should be either 'True' if it was, or 'False' if it was not.
    ### Input:
    [Critic] Frank Swietek
    [Review] 
    ### Response:
    True
    """
    if review['critic'] == critic:
        return


Real Label:Ben and J-Lo are no Tracy and Hepburn--they're more like the Sean Penn and Madonna of Shanghai Surprise. And their movie is just as rotten as that one.


--------------------------------------------------

Generated Review 2:
Below is a piece of review content. Your task is to verify whether the review was written by the critic specified. The response should always be True if the review was written by the critic, or False if it was not.
    

# Test Model2

In [23]:
from peft import PeftModel

MODEL = "model2"
adapters_name = f"experiments/generator/{MODEL}"
BASE_MODEL = "decapoda-research/llama-7b-hf"

generator_model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
generator_model = PeftModel.from_pretrained(generator_model, adapters_name)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [24]:
test_dataset_data = train_test_datasets[MODEL]['test']

In [25]:
responses, real_comments = generate_reviews(MODEL, generator_model, test_dataset_data, tokenizer, GENERATE_CASE, PRINT_CASE)

Generated Review 1:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
Write a new review for Gigli
### Input:
[Movie_info] Gigli (Ben Affleck) is ordered to kidnap the psychologically challenged younger brother of a powerful federal prosecutor. When plans go awry, Gigli's boss sends in Ricki (Jennifer Lopez), a gorgeous free-spirited female gangster who has her own set of orders to assist with the kidnapping. But Gigli begins falling for the decidedly unavailable Ricki, which could be a hazard to his occupation.
[Movie genres]Action & Adventure, Comedy, Drama, Romance
[Review score]F
[review history]["[REVIEW] Has a derivative feel, overly reminiscent not only of fifties Broadway drama but of some of Allen's previous pictures...while it probably won't bore you, it's likely to leave you with a feeling of déjà vu.", "[REVIEW] Astonishingly bad...a wou

EVAL

In [26]:
sentiment_scores = []
for i in range(len(responses)):
    score = evaluate_sentiment_match(real_comments[i],responses[i])
    sentiment_scores.append(score)

print("sentiment_scores:",sum(sentiment_scores) / len(sentiment_scores))
print("cossim:",evaluate_cossim(responses,real_comments))

sentiment_scores: 0.5766169999999998
cossim: 0.2984967733081794


Discriminator(generator)

In [27]:
test_prompts = []
for i in range(PRINT_CASE):
    temp_df = df[df["review_content"] == real_comments[i]]
    temp_df["fake_review"] = responses[i]
    test_prompts.append(generate_discriminator_test_prompt(temp_df, fake=True))

<ipython-input-27-09a91492e7fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["fake_review"] = responses[i]
<ipython-input-27-09a91492e7fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["fake_review"] = responses[i]
<ipython-input-27-09a91492e7fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [28]:
#set config for model generation
generation_config = discriminator_model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

# test case
for i in range(PRINT_CASE):
    input_ids = tokenizer.encode(test_prompts[i], return_tensors='pt')
    output = discriminator_model.generate(input_ids=input_ids,
                        generation_config=generation_config)
    #decode output
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f"Generated Review {i+1}:\n{output_str}\n\n")
    print(f"Real Label:{real_comments[i]}\n")
    print("\n"+"-"*50+"\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated Review 1:
Below is a piece of review content. Your task is to verify whether the review was written by the critic specified. The response should always be True if the review was written by the critic, or False if it was not.
    ### Instruction:
    Verify if the review content was written by the specified critic. Your response should be either 'True' if it was, or 'False' if it was not.
    ### Input:
    [Critic] Frank Swietek
    [Review] [Movie_info] Gigli (Ben Affleck) is ordered to kidnap the psychologically challenged younger brother of a powerful federal prosecutor. When plans go awry, Gigli's boss sends in Ricki (Jennifer Lopez), a g
    ### Response:
    True
    ### Expected Output:
    True
    ### Actual Output:


Real Label:Ben and J-Lo are no Tracy and Hepburn--they're more like the Sean Penn and Madonna of Shanghai Surprise. And their movie is just as rotten as that one.


--------------------------------------------------

Generated Review 2:
Below is a piece

# Test Model3

In [29]:
from peft import PeftModel

MODEL = "model3"
adapters_name = f"experiments/generator/{MODEL}"
BASE_MODEL = "decapoda-research/llama-7b-hf"

generator_model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
generator_model = PeftModel.from_pretrained(generator_model, adapters_name)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [30]:
test_dataset_data = train_test_datasets[MODEL]['test']

In [31]:
responses, real_comments = generate_reviews(MODEL, generator_model, test_dataset_data, tokenizer, GENERATE_CASE, PRINT_CASE)

Generated Review 1:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
You are a movie critic, using movie infos and review you have written before, write a new short review for movie Gigli
### Input:
[Movie_info] Gigli (Ben Affleck) is ordered to kidnap the psychologically challenged younger brother of a powerful federal prosecutor. When plans go awry, Gigli's boss sends in Ricki (Jennifer Lopez), a gorgeous free-spirited female gangster who has her own set of orders to assist with the kidnapping. But Gigli begins falling for the decidedly unavailable Ricki, which could be a hazard to his occupation.
[Movie genres]Action & Adventure, Comedy, Drama, Romance
[Review score]F
[review history]["[REVIEW] Has a derivative feel, overly reminiscent not only of fifties Broadway drama but of some of Allen's previous pictures...while it probably won't bore you,

EVAL

In [32]:
sentiment_scores = []
for i in range(len(responses)):
    score = evaluate_sentiment_match(real_comments[i],responses[i])
    sentiment_scores.append(score)

print("sentiment_scores:",sum(sentiment_scores) / len(sentiment_scores))
print("cossim:",evaluate_cossim(responses,real_comments))

sentiment_scores: 0.5465359999999999
cossim: 0.29297446672837396


Discriminator(generator)

In [33]:
test_prompts = []
for i in range(PRINT_CASE):
    temp_df = df[df["review_content"] == real_comments[i]]
    temp_df["fake_review"] = responses[i]
    test_prompts.append(generate_discriminator_test_prompt(temp_df, fake=True))

<ipython-input-33-09a91492e7fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["fake_review"] = responses[i]
<ipython-input-33-09a91492e7fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["fake_review"] = responses[i]
<ipython-input-33-09a91492e7fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [34]:
#set config for model generation
generation_config = discriminator_model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

# test case
for i in range(PRINT_CASE):
    input_ids = tokenizer.encode(test_prompts[i], return_tensors='pt')
    output = discriminator_model.generate(input_ids=input_ids,
                        generation_config=generation_config)
    #decode output
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f"Generated Review {i+1}:\n{output_str}\n\n")
    print(f"Real Label:{real_comments[i]}\n")
    print("\n"+"-"*50+"\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated Review 1:
Below is a piece of review content. Your task is to verify whether the review was written by the critic specified. The response should always be True if the review was written by the critic, or False if it was not.
    ### Instruction:
    Verify if the review content was written by the specified critic. Your response should be either 'True' if it was, or 'False' if it was not.
    ### Input:
    [Critic] Frank Swietek
    [Review] Gigli is a movie that is a derivative feel, overly reminiscent not only of fifties Broadway drama but of some of Allen's previous pictures. While it probably won't bore you, it's likely to leave you with a feeling of déjà vu.
G
    ### Response:
    True
    ### Explanation:
    The review was written by Frank Swiet


Real Label:Ben and J-Lo are no Tracy and Hepburn--they're more like the Sean Penn and Madonna of Shanghai Surprise. And their movie is just as rotten as that one.


--------------------------------------------------

Generat